# TP8 - Cartographie sous `python`

#### Analyse de Données Massives - Master 1ère année


## Exercices

Nous allons utiliser la base de données des restaurants new-yorkais présente sur le serveur MongoDB, vu dans le TP précédent. Sur cette base, vous devez faire les demandes suivantes :

### 1. Afficher un marqueur pour chaque restaurant, sur la carte de New-York

Dans un premier temps, on importe les données des restaurants, via le connecteur à MongoDB. Ici, on fait attention à ne pas avoir de restaurants sans coordonnées (cela fait planter `folium` sinon).

In [1]:
import pymongo
import pandas
con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")
db = con.test.restaurants

res = db.find({ "address.coord": { "$ne": [] }})

Ensuite, on transforme ce résultat en un `DataFrame`.

In [2]:
restos = pandas.DataFrame(list(res))

restos.head()

,_id,address,borough,cuisine,grades,name,restaurant_id
0,58ac16d1a251358ee4ee87dd,"{'zipcode': '10462', 'coord': [-73.856077, 40....",Bronx,Bakery,"[{'score': 2, 'grade': 'A', 'date': 2014-03-03...",Morris Park Bake Shop,30075445
1,58ac16d1a251358ee4ee87de,"{'zipcode': '11225', 'coord': [-73.961704, 40....",Brooklyn,Hamburgers,"[{'score': 8, 'grade': 'A', 'date': 2014-12-30...",Wendy'S,30112340
2,58ac16d1a251358ee4ee87df,"{'zipcode': '10019', 'coord': [-73.98513559999...",Manhattan,Irish,"[{'score': 2, 'grade': 'A', 'date': 2014-09-06...",Dj Reynolds Pub And Restaurant,30191841
3,58ac16d1a251358ee4ee87e0,"{'zipcode': '11224', 'coord': [-73.98241999999...",Brooklyn,American,"[{'score': 5, 'grade': 'A', 'date': 2014-06-10...",Riviera Caterer,40356018
4,58ac16d1a251358ee4ee87e1,"{'zipcode': '11374', 'coord': [-73.8601152, 40...",Queens,Jewish/Kosher,"[{'score': 20, 'grade': 'Z', 'date': 2014-11-2...",Tov Kosher Kitchen,40356068


Ci-dessous, nous voyons comment nous pouvons avoir les coordonnées d'un restaurant (le premier ici). Notre problème est que celles-ci sont au format $latitude, longitude$, alors que la fonction `folium.Marker()` demande l'inverse ($lat, long$).

In [3]:
restos.loc[0,"address"]["coord"]

[-73.856077, 40.848447]

A l'aide de l'instruction `[::-1]`, nous pouvons avoir l'inverse d'une liste, ce que nous souhaitons ici.

In [4]:
restos.loc[0,"address"]["coord"][::-1]

[40.848447, -73.856077]

Nous pouvons donc maintenant créer une carte avec un marqueur pour chaque restaurant (ici, nous nous restreignons à 100 restaurants). 

In [5]:
import folium

centre = [40.71427, -74.00597]
carte1a = folium.Map(location = centre, zoom_start = 10)
for i in range(100): # range(len(restos)):
    folium.Marker(restos.loc[i,"address"]["coord"][::-1]).add_to(carte1a)
carte1a

Malheureusement, le nombre de restaurants est trop important. Il nous faut donc passer par des groupes de marqueurs (cf [`MarkerCluster()`](https://python-visualization.github.io/folium/plugins.html#folium.plugins.MarkerCluster)). 

In [6]:
from folium.plugins import MarkerCluster

carte1b = folium.Map(location = centre, zoom_start = 10)
groupes = MarkerCluster().add_to(carte1b)
for i in range(5000): # range(len(restos)):
    folium.Marker(restos.loc[i,"address"]["coord"][::-1]).add_to(groupes)
carte1b

Mais ceci ne fonctionne que jusqu'à environ 5000 marqueurs environ. Après, le rendu de la carte n'est pas effectuée. Nous allons utiliser le plugin [`FastMarkerCluster()`](https://python-visualization.github.io/folium/plugins.html#folium.plugins.FastMarkerCluster). Pour l'utliser, nous devons lui passer une liste contenant les coordonnées $lat, long$ dans une sous-liste. 

In [7]:
liste = list(
    zip(
        [r["coord"][1] for r in restos["address"]], 
        [r["coord"][0] for r in restos["address"]]
    )
)
liste[:10]


[(40.848447, -73.856077),
 (40.662942, -73.961704),
 (40.7676919, -73.98513559999999),
 (40.579505, -73.98241999999999),
 (40.7311739, -73.8601152),
 (40.7643124, -73.8803827),
 (40.6119572, -74.1377286),
 (40.6199034, -73.9068506),
 (40.628886, -74.00528899999999),
 (40.6408271, -73.9482609)]

In [8]:
from folium.plugins import FastMarkerCluster

carte1c = folium.Map(location = centre, zoom_start = 10)
FastMarkerCluster(liste).add_to(carte1c)
carte1c

### 2. Ajouter à ces marqueurs une popup intégrant le nom du restaurant

On doit utiliser soit `Marker()`, soit `MarckerCluster()`. La fonction `FastMarkerCluster()` ne permet pas de personnaliser les marqueurs.

In [9]:
import html

carte2 = folium.Map(location = centre, zoom_start = 10)
for i in range(100): # range(len(restos)):
    folium.Marker(restos.loc[i, "address"]["coord"][::-1], popup = html.escape(restos.loc[i, "name"])).add_to(carte2)
carte2

### 3. Améliorer ces popups, en y ajoutant le quartier

Nous définissons une fonction prenant en paramètre toutes les informations d'un restaurant, et qui renvoie une chaîne de caractères comprenant le nom et le quartier.

In [10]:
def texte(resto):
    return html.escape(resto["name"] + ' (' + resto["borough"] + ')')

carte3 = folium.Map(location = centre, zoom_start = 10)
for i in range(100): # range(len(restos)):
    folium.Marker(restos.loc[i, "address"]["coord"][::-1], popup = texte(restos.loc[i,])).add_to(carte3)
carte3

### 4. Améliorer encore en mettant en gras le nom, en italique le quartier, et en ajoutant l'adresse, ainsi que le nombre d'évaluation pour chaque score

Maintenant, nous redéfinissons la fonction pour renvoyer le code HTML permettant d'avoir l'affichage demandé.

In [11]:
def texte(resto):
    tab = pandas.DataFrame(resto["grades"])[["grade", "score"]].groupby("grade").count()
    return "<strong>" + html.escape(resto["name"]) + '</strong> (<em>' + html.escape(resto["borough"]) + '</em>)<br>' +\
            resto["address"]["building"] + " " + html.escape(resto["address"]["street"]) + "<br>" +\
            tab.to_html()

carte4 = folium.Map(location = centre, zoom_start = 10)
for i in range(100): # range(len(restos)):
    folium.Marker(restos.loc[i, "address"]["coord"][::-1], popup = texte(restos.loc[i,])).add_to(carte4)
carte4

### 5. Afficher les quartiers sur la carte de New-York, avec une couleur dépandant du nombre de restaurants présents dans le quartier

Dans un premier temps, nous devons calculer le nombre de restaurants par quartier.

In [12]:
import json
geo = json.load(open("newyork-borough.geojson"))

res = db.aggregate([
    { "$match": { "borough": { "$ne": "Missing" }}},
    { "$group": { "_id": "$borough", "nb": { "$sum": 1 }}},
    { "$project": { "Quartier": "$_id", "Nb restos": "$nb", "_id": 0 }}
])
df = pandas.DataFrame(list(res))
df

,Nb restos,Quartier
0,969,Staten Island
1,6086,Brooklyn
2,2338,Bronx
3,5656,Queens
4,10259,Manhattan


Ensuite, nous pouvons créer la carte choroplèthe, avec une couleur en fonction du nombre de restaurants.

In [13]:
carte5 = folium.Map(location = centre, zoom_start = 10)
carte5.choropleth(geo_data = geo, key_on = "feature.properties.BoroName",
                  data = df, columns = ["Quartier", "Nb restos"],
                  fill_color = 'YlGn',
                  threshold_scale = [0, 2500, 5000, 7500, 10000, 12500],
                  legend_name = "Nombre de restaurants par quartier")
carte5

### 6. Faire la même carte, mais avec une couleur en fonction du score moyen

Il faut seulement adapter le calcul à faire dans Mongo. Le reste est identique.

In [14]:
res = db.aggregate([
    { "$match": { "borough": { "$ne": "Missing" }}},
    { "$unwind": "$grades" },
    { "$group": { "_id": "$borough", "nb": { "$avg": "$grades.score" }}},
    { "$project": { "Quartier": "$_id", "Score moyen": "$nb", "_id": 0}}
])
df = pandas.DataFrame(list(res))
df

,Quartier,Score moyen
0,Staten Island,11.370958
1,Brooklyn,11.447976
2,Bronx,11.036186
3,Queens,11.634865
4,Manhattan,11.418151


In [15]:
carte5 = folium.Map(location = centre, zoom_start = 10)
carte5.choropleth(geo_data = geo, key_on = "feature.properties.BoroName",
                  data = df, columns = ["Quartier", "Score moyen"],
                  fill_color = 'OrRd',
                  legend_name = "Score par quartier")
carte5